In [164]:
import numpy as np
import pandas as pd
import json
import requests

In [165]:
from eric_keys import eric_omdb_key, eric_tmdb_key, eric_tmdb_token
from eric_test import *
from functions import *

In [167]:
# Open TMDB Details DF
df_det = pd.read_csv('TMDB_movie_details.csv')
print(len(df_det))
print(df_det.keys())

7132
Index(['id', 'imdb_id', 'budget', 'revenue', 'genre_names', 'original_title',
       'production_countries', 'runtime', 'status'],
      dtype='object')


In [168]:
# Reformat TMDB Details Contries
import ast
df_det['countries']=df_det['production_countries'].apply(lambda x: ast.literal_eval(x))
df_det.drop('production_countries', axis=1, inplace=True)

In [234]:
df_top = pd.read_csv('clean_TMDB_toprated.csv', index_col='title').sort_index()
df_top

,id,original_language,avg_vote,vote_count,popularity,genres,release_date
title,,,,,,,
10 Days without Mamma,575100,it,6.6,217,8.402,[35],2019-02-07
13 Assassins,58857,ja,7.3,593,10.583,"[28, 12, 18]",2010-09-09
17 Girls,87368,fr,5.5,214,5.017,[18],2011-06-13
1900,3870,it,7.9,269,10.527,"[18, 36, 10749]",1976-09-01
2046,844,zh,7.2,417,10.868,"[18, 14, 878, 10749]",2004-05-20
...,...,...,...,...,...,...,...
Zombie Flesh Eaters,7219,it,6.6,289,7.412,[27],1979-08-25
[REC],8329,es,7.1,2367,18.778,"[27, 9648]",2007-04-10
[REC]²,10664,es,6.5,1110,19.205,"[27, 53]",2009-09-15
